In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Conv1D, Activation, GlobalMaxPool1D, Dense
from keras import Input, optimizers

file_path = '/Users/pratiktale/Research Project/dataset_malwares.csv'
df = pd.read_csv(file_path)

selected_features = df[["MinorOperatingSystemVersion", "MajorSubsystemVersion",
                        "SizeOfStackReserve", "MajorLinkerVersion",
                        "MajorOperatingSystemVersion", "MinorImageVersion",
                        "Subsystem", "MinorSubsystemVersion"]]
labels = df["Malware"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

max_size = len(selected_features.columns)
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, max_size))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, max_size))

inputs = Input(shape=(1, max_size))
conv = Conv1D(kernel_size=3, filters=32, strides=1, padding='same')(inputs)
act = Activation('relu')(conv)
p = GlobalMaxPool1D()(act)
d = Dense(16, activation='relu')(p)
predictions = Dense(1, activation='sigmoid')(d)

model = Model(inputs=inputs, outputs=predictions)

my_opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=my_opt, loss="binary_crossentropy", metrics=["acc"])

history = model.fit(X_train_scaled, y_train, batch_size=32, epochs=20, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

model.save('my_malconv_model.h5')

history_df = pd.DataFrame(history.history)
history_df.index.name = 'Epoch'
print("\nTraining and Validation Metrics:")
print(history_df.to_markdown())


Epoch 1/20
393/393 [==============================] - 1s 708us/step - loss: 0.3026 - acc: 0.9225 - val_loss: 0.1863 - val_acc: 0.9395
Epoch 2/20
393/393 [==============================] - 0s 714us/step - loss: 0.1726 - acc: 0.9467 - val_loss: 0.1703 - val_acc: 0.9452
Epoch 3/20
393/393 [==============================] - 0s 577us/step - loss: 0.1595 - acc: 0.9529 - val_loss: 0.1598 - val_acc: 0.9516
Epoch 4/20
393/393 [==============================] - 0s 578us/step - loss: 0.1501 - acc: 0.9572 - val_loss: 0.1502 - val_acc: 0.9528
Epoch 5/20
393/393 [==============================] - 0s 572us/step - loss: 0.1417 - acc: 0.9591 - val_loss: 0.1439 - val_acc: 0.9563
Epoch 6/20
393/393 [==============================] - 0s 569us/step - loss: 0.1351 - acc: 0.9618 - val_loss: 0.1351 - val_acc: 0.9602
Epoch 7/20
393/393 [==============================] - 0s 595us/step - loss: 0.1302 - acc: 0.9631 - val_loss: 0.1294 - val_acc: 0.9627
Epoch 8/20
393/393 [==============================] - 0s 598us

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
